### 1. 패키지 설치

In [ ]:
pip install --upgrade pip

In [ ]:
pip install -r requirements.txt  # 패키지 설치

In [ ]:
import torch
print(torch.__version__)  # PyTorch 버전
print(torch.version.cuda)  # PyTorch가 사용하는 CUDA 버전
print(torch.cuda.is_available())  # GPU 사용 가능 여부


### 2. 문서 split 및 Chroma를 활용한 vector store 구성

In [1]:
from huggingface_hub import whoami

try:
    user_info = whoami()
    print(f"로그인 상태입니다. 사용자: {user_info['name']}")
except Exception as e:
    print("로그인되지 않았거나 토큰이 유효하지 않습니다.")
    print(e)

/home/chae/env_0217/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


로그인 상태입니다. 사용자: chaeeee


In [2]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "false"

In [ ]:
from langchain.schema import Document
import os, json
from tqdm import tqdm

# 1. JSON 파일 경로 설정
json_path = "./json_data.json"  # 단일 JSON 파일 경로

# 2. JSON 데이터 불러오기
with open(json_path, 'r', encoding='utf-8') as f:
    all_data = json.load(f)

# 3. {}를 기준으로 JSON 데이터 분할 및 Document 객체 생성
documents = []

for data in tqdm(all_data, desc="Generating Documents", unit="entry", ncols=80):
    metadata = {
        "title": data.get('title', 'N/A'),
        "artist": data.get('artist', 'N/A'),
        "year": data.get('year', 'N/A'),
        "read_count": data.get('read_count', 0)
    }

    # JSON 데이터의 각 항목을 Document 객체로 변환
    doc_content = json.dumps(data, ensure_ascii=False, indent=4)
    documents.append(Document(
        page_content=doc_content.strip(),
        metadata=metadata
    ))

In [ ]:
len(documents)

In [ ]:
documents[1]

In [ ]:
# 특정 Document 객체의 텍스트 길이 확인
len(documents[11001].page_content)

In [ ]:
# 800자를 초과하는 Document 개수 세기
over_800_count = sum(1 for doc in documents if len(doc.page_content) > 800)

# 결과 출력
print(f"800자를 초과하는 Document 개수: {over_800_count}")


In [3]:
from langchain.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
import faiss  # FAISS 라이브러리 필요

# 1. 임베딩 초기화
embedding_model = SentenceTransformer("nlpai-lab/KURE-v1")


You try to use a model that was created with version 3.3.1, however, your version is 3.2.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [ ]:
# 2. 문서 데이터와 메타데이터 분리
texts = [doc.page_content for doc in documents]  # 문서 텍스트
metadatas = [doc.metadata for doc in documents]  # 문서 메타데이터

# 3. 문서 임베딩 생성
embeddings = embedding_model.encode(texts)

In [ ]:
# 4. FAISS 인덱스 생성
embedding_dim = embeddings.shape[1]  # 벡터 차원 확인
faiss_index = faiss.IndexFlatL2(embedding_dim)  # L2 거리 기반 인덱스
faiss_index.add(embeddings)  # 벡터 추가


In [ ]:
from langchain.docstore.in_memory import InMemoryDocstore

# 5. Docstore 생성
# 각 문서에 고유 ID를 부여해 InMemoryDocstore 생성
docstore = InMemoryDocstore({i : doc for i, doc in enumerate(documents)})

In [ ]:
# 6. FAISS 벡터스토어 생성
def embed_query(text):
    return embedding_model.encode([text])[0]  # 단일 쿼리 텍스트를 임베딩

index_to_docstore_id = {i: i for i in range(len(documents))}

faiss_db = FAISS(
    embedding_function=embed_query,
    index=faiss_index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)

# 7. FAISS 데이터베이스 저장
faiss_db.save_local("./faiss_artworks_0304_artworks")
print("FAISS 데이터베이스가 성공적으로 저장되었습니다!")


In [ ]:
# 9. 검색 테스트
query = "박승무의 설경"
results = faiss_db.similarity_search(query, k=5)

# 10. 검색 결과 출력
for result in results:
    print("문서 텍스트:", result.page_content)
    print("문서 메타데이터:", result.metadata)
    print('=------------------------------------------')


In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("nlpai-lab/KURE-v1")

In [4]:
from langchain_community.vectorstores import FAISS


# 기존 DB 로드 
persist_directory = "./faiss_combined"

try:
    faiss_db = FAISS.load_local(
        folder_path=persist_directory,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True  # 신뢰할 수 있는 소스에서만 사용
    )
    
    # embedding_function 수정
    faiss_db.embedding_function = lambda text: (
        embedding_model.encode(text) if isinstance(text, str) else embedding_model.encode(str(text))
    )
    
    print("FAISS 데이터베이스가 성공적으로 로드되었습니다!")
except Exception as e:
    print(f"FAISS 데이터베이스 로드 중 오류 발생: {e}")

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


FAISS 데이터베이스가 성공적으로 로드되었습니다!


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4bit 양자화 활성화
    bnb_4bit_compute_dtype=torch.float16,  # 계산 타입 설정 (float16이 일반적)
    bnb_4bit_use_double_quant=True,  # 더블 양자화 사용 (메모리 절약)
    bnb_4bit_quant_type="nf4",  # NormalFloat4 (NF4) 사용
)

In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit 양자화 활성화
    bnb_4bit_compute_dtype="float16",  # 계산 정밀도 설정
    bnb_4bit_quant_type="nf4",  # NF4 양자화 방식 사용 (효율적)
    bnb_4bit_use_double_quant=True,  # 이중 양자화 사용
)

In [ ]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# 모델과 토크나이저 로드 (CUDA 사용)
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,  # ✅ 올바른 양자화 설정 적용
    device_map="auto",  # ✅ 자동 GPU 배치
    trust_remote_code=True,
)

In [ ]:
from transformers import pipeline

# 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,  # 생성할 최대 토큰 수 증가
    do_sample=True,        # 샘플링 활성화
    temperature=0.1,      
    top_k=50,             
    repetition_penalty=1.05
)
# LangChain의 HuggingFacePipeline 사용
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
from langchain.prompts import ChatPromptTemplate

deepseek_template = """
<|system|>
You are a friendly chatbot specializing in artworks and general conversations.
Your primary role is to answer questions **accurately based on the provided document (context)**. 
If the requested information is not found in the document, respond with:
"문서에 해당 정보가 없습니다." 

However, if the question is a general conversation or does not relate to the document, you should respond naturally as a conversational chatbot. 
You can talk about art history, artists, exhibitions, and general topics such as daily life, technology, and culture. 
Maintain a friendly and engaging tone, ensuring all responses are written in Korean.
Use **beautiful Markdown formatting** (headings, bullet points, **bold** or *italic* text) to enhance readability.
You must include the artwork number in your response.

<|context|>
{context}

<|user|>
Question: {question}

<|assistant|>
"""




exaone_template = '''
<|system|>
You are an AI assistant tasked with refining and polishing the provided logical reasoning into a final answer in Korean.  
Your role is to produce a clear, concise, and well-structured response that maintains the original meaning and key details.  
Ensure that your final answer is written in Korean and uses **beautiful Markdown formatting** (e.g., headings, bullet points, **bold** or *italic* text) to enhance readability.  
Focus solely on refining the content without adding any new information.
You must include the artwork number in your response.

<|reasoning|>
{reasoning}

<|user|>
Based on the above reasoning, please generate a refined and final answer in Korean.

<|assistant|>
'''



# DeepSeek 템플릿 생성
deepseek_prompt = ChatPromptTemplate.from_template(deepseek_template)

# EXAONE 템플릿 생성
exaone_prompt = ChatPromptTemplate.from_template(exaone_template)



In [14]:
from langchain.prompts import ChatPromptTemplate

template = '''
<|system|>
You are a friendly chatbot specializing in artworks and general conversations.
Your primary role is to answer questions strictly based on the information provided in the document (context). 
If the requested information is not found in the document, respond with:
"The document does not contain this information." 

However, if the question is a general conversation or does not relate to the document, you should respond naturally as a conversational chatbot. 
You can talk about art history, artists, exhibitions, and general topics such as daily life, technology, and culture. 
Maintain a friendly and engaging tone, ensuring all responses are written in Korean.
Use **beautiful Markdown formatting** (headings, bullet points, bold or italic text) to enhance readability.
You must include artwork number.

<|context|>
{context}

<|user|>
Question: {question}

<|assistant|>
'''

# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template(template)


In [8]:
retriever = faiss_db.as_retriever(
    search_kwargs={
        "k": 5,                # 검색 결과 개수
        "fetch_k": 15,         # 더 많은 결과 가져오기
        "mmr": True,           # MMR 활성화
        "mmr_beta": 0.3      # 다양성과 관련성 간 균형
    }
)


In [9]:
import re

class MarkdownOutputParser:
    """Enhanced Markdown parser with additional formatting options."""

    def __call__(self, llm_output):
        """Extracts the assistant's response from after the </think> tag and formats it in Markdown."""
        if not llm_output or llm_output.strip() == "":
            return "❌ 모델에서 응답을 생성하지 못했습니다."

        # "</think>" 이후 텍스트 추출
        match = re.search(r"</think>\s*(.*)", llm_output, re.DOTALL)
        extracted_text = match.group(1).strip() if match else llm_output.strip()

        # Markdown 형식 적용
        formatted_output = f"""
### **🔹 모델 응답 결과**

{extracted_text}
"""
        return formatted_output.strip()  # 양 끝 공백 제거


In [21]:
import re

class MarkdownOutputParser2:
    """Enhanced Markdown parser with additional formatting options."""

    def __call__(self, llm_output):
        """Extracts the assistant's response from after the </think> tag and formats it in Markdown."""
        if not llm_output or llm_output.strip() == "":
            return "❌ 모델에서 응답을 생성하지 못했습니다."

        # "</think>" 이후 텍스트 추출
        match = re.search(r"<\|assistant\|>\s*(.*)", llm_output, re.DOTALL)
        extracted_text = match.group(1).strip() if match else llm_output.strip()

        # Markdown 형식 적용
        formatted_output = f"""
### **🔹 모델 응답 결과**

{extracted_text}
"""
        return formatted_output.strip()  # 양 끝 공백 제거


In [10]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# 🔹 EXAONE 모델 로드
exaone_model_id = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"
exaone_tokenizer = AutoTokenizer.from_pretrained(exaone_model_id)
exaone_model = AutoModelForCausalLM.from_pretrained(
    exaone_model_id,
    quantization_config=quantization_config,
    device_map="cuda",  # CUDA에서 자동 배치
    trust_remote_code=True
)


Loading checkpoint shards: 100%|██████████| 7/7 [00:23<00:00,  3.43s/it]


In [11]:
from transformers import pipeline

# 파이프라인 생성
exaone_pipe = pipeline(
    "text-generation",
    model=exaone_model,
    tokenizer=exaone_tokenizer,
    max_new_tokens=1024,  # 생성할 최대 토큰 수 증가
    do_sample=True,        # 샘플링 활성화
    temperature=0.1,      
    top_k=50,             
    repetition_penalty=1.05
)
# LangChain의 HuggingFacePipeline 사용
exaone_llm = HuggingFacePipeline(pipeline=exaone_pipe)

/tmp/ipykernel_1464/2480121385.py:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  exaone_llm = HuggingFacePipeline(pipeline=exaone_pipe)


In [22]:
from langchain.schema.runnable import RunnableLambda
from langchain.schema.runnable import RunnablePassthrough, RunnableMap

chain = (
    RunnableMap({
        "context": retriever,               # Retriever에서 반환된 값을 가져옴
        "question": RunnablePassthrough()   # 질문은 그대로 전달
    })
    | (lambda x: {
        "context": "\n".join([doc.page_content for doc in x["context"]]),
        "question": x["question"]
    })  # context를 문자열로 변환
    | prompt                               # Prompt Template에 전달
    | exaone_llm                                  # LLM으로 응답 생성
    | MarkdownOutputParser2()                    # 응답을 문자열로 변환
)

In [ ]:
from langchain.schema.runnable import RunnableLambda

chain = (
    retriever
    | RunnableLambda(lambda docs: {  
        "context": "\n".join([doc.page_content for doc in docs]),  
        "question": query 
    })
    | deepseek_prompt
    | llm
    | MarkdownOutputParser()
    | (lambda x: {"reasoning": x})
    | exaone_prompt
    | exaone_llm
    | MarkdownOutputParser2()
)


In [36]:
query = "양혜규의 신용할만한 산과 굴절에 대해 알려줘."

In [37]:
response = chain.invoke({"question": query})
print(response)

### **🔹 모델 응답 결과**

## 양혜규의 작품: 신용할 만한 산과 굴절 (#13, #14, #17, #22, #27, #28, #30)

**개요:**

양혜규의 작품 **신용할 만한 산과 굴절** 은 2010년 제작된 드로잉 시리즈입니다. 오스트리아 브레겐츠 미술관에서 2011년 개인전을 통해 처음 공개되었습니다. 이 작품은 **금융 정보를 담은 우편물 봉투** 를 재해석하여 제작되었습니다. 

**핵심 개념:**

* **개념적 전환 (Détournement):**  일상적인 우편물 봉투에 숨겨진 의미를 드러내고 새로운 맥락을 부여합니다.
* **소통과 정보:** 금융 정보를 담은 우편물을 통해 현대 사회에서 이루어지는 소통 방식과 정보의 흐름에 대한 질문을 던집니다.
* **재활용과 예술:** 버려질 운명이었던 우편물 봉투를 예술 작품으로 재탄생시켜 물질의 새로운 가능성을 제시합니다.

**구성:**

* **개별 작품:**  <신용할 만한 산 #22>, <신용할 만한 산 #17>, <신용할 만한 산 #30>, <신용할 만한 굴절 #28>, <신용할 만한 굴절 #13>, <신용할 만한 굴절 #14>, <신용할 만한 굴절 #27> 로 구성됩니다. 각 작품은 번호로 구분되며, 전시장 공간과 상호작용하도록 설치됩니다.
* **재료:** 카드보드에 보안 편지 봉투, 모눈종이, 콜라주가 주요 재료로 사용됩니다. 기하학적 패턴과 색상의 조화를 통해 시각적 효과를 창출합니다.

**의미:**

양혜규는 이 작품을 통해 다음과 같은 메시지를 전달합니다.

* **일상 속 의미 찾기:** 평범한 물건 속에 숨겨진 의미와 가치를 발견하고 재해석하는 과정을 보여줍니다.
* **물질의 순환:** 버려지는 물질에 예술적 생명력을 불어넣어 지속 가능한 사회에 대한 고찰을 제시합니다.
* **정보 시대의 소통:** 금융 정보를 매개로 한 현대 사회의 소통 방식과 그 이면에 숨겨진 문제점을 질문합니다.


In [38]:
retrieved_docs = retriever.get_relevant_documents(query)
for i, doc in enumerate(retrieved_docs):
    print(f"Document {i+1}:")
    print(f"Content: {doc.page_content}")  # 문서의 실제 내용
    print(f"Metadata: {doc.metadata}")    # 메타데이터 (예: 출처, 페이지 등)
    print("-" * 50)


Document 1:
Content: 작품명: 신용할 만한 산과 굴절 #13, #14, #17, #22, #27, #28, #30 / N/A / Trustworthy Mountains and Refractions #13, #14, #17, #22, #27, #28, #30

작가: 양혜규 / YANG Haegue

작품 번호: 8088

제작 연도: 2010

크기: 99×69×(7)

재료: 카드보드에 보안 편지 봉투, 모눈종이, 콜라주

카테고리: 드로잉

작품 설명: 양혜규(1971- )는 서울대 조소과를 졸업하고 독일 프랑크푸르트 국립학교 슈테델슐레에서 마이스터슐러 학위를 취득했다. 1994년이래 프랑크푸르트, 베를린, 서울을 기반으로 국제 미술무대에서 왕성하게 활동하며 대규모 설치, 조각, 평면 등 다양한 매체를 아우르는 작업을 선보여왔다.<신용할 만한 산과 굴절 #13, #14, #17, #22, #27, #28, #30>(2010)은 2011년 오스트리아 브레겐츠 미술관에서 선보였던 버전으로 <신용할 만한 산 #22>, <신용할 만한 산 #17>, <신용할 만한 산 #30>, <신용할 만한 굴절 #28>, <신용할 만한 굴절 #13>, <신용할 만한 굴절 #14>, <신용할 만한 굴절 #27>의 구성으로 금융 정보를 담은 우편물 봉투의 개념적 전환(détournement)에 집중한 초기 작업이라 할 수 있다. 은행 카드의 비밀번호 등, 금융 정보를 담은 우편물이 담겨 오는 편지 봉투의 내지를 주 재료로 한 꼴라쥬를 통해, 일상에서 보이는 혹은 가려진 소통에 대한 물음을 던진다. 꼴라주로 조형된 평면적인 이미지는 기하, 대칭, 그리고 색상의 단계적 차이 등 시각적 효과를 탐구한 결과물인 것이다. 작가는 산업 생산물인 편지 봉투의 수명 주기에 대해서 이야기 하고 있는데, 여기에는 편지 봉투의 이동 거리, 기능, 소재 및 다양한 디자인 패턴 등이 포함된다. 이제 작가는 이제껏 가려지거나 보호되었던 편지 봉투 안의 메시지들을 재조합함으로써, 편지 봉투의 

/tmp/ipykernel_1464/3695306690.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


In [ ]:
# 검색 수행: 유사도 점수와 함께 반환
docs_and_scores = retriever.vectorstore.similarity_search_with_score(query, k=5)

# 검색된 문서 수 출력
print(f"검색된 문서 수: {len(docs_and_scores)}")

# 각 문서의 파일명, 전체 내용, 유사도 점수 출력
for i, (doc, score) in enumerate(docs_and_scores, 1):
    print(f"\n문서 {i}:")
    print(f"  파일명: {doc.metadata.get('source', 'N/A')}")
    print(f"  유사도 점수: {score:.4f}")
    print(f"  전체 내용: {doc.page_content}")


### PDF DB에 추가

In [ ]:
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import fitz  # PyMuPDF
import pdfplumber
from langchain_core.documents import Document
from langchain.docstore.in_memory import InMemoryDocstore

# 1. 임베딩 모델 초기화
embedding_model = SentenceTransformer("nlpai-lab/KURE-v1")

# 2. FAISS 인덱스 로드 및 L2 거리 기반 인덱스 사용
def load_existing_faiss_db(index_path, embedding_dim):
    try:
        faiss_index = faiss.read_index(index_path)  # 기존 FAISS DB 로드
        print(f"FAISS index loaded from {index_path}.")
    except Exception as e:
        print(f"Error loading FAISS index from {index_path}. Error: {e}")
        # L2 거리 기반 인덱스 새로 생성 (embedding_dim: 벡터 차원)
        faiss_index = faiss.IndexFlatL2(embedding_dim)
        print("새로운 FAISS L2 인덱스 생성.")
    return faiss_index


# 3. PDF에서 텍스트와 표를 문단별로 추출
def extract_text_paragraphs(pdf_path):
    """
    PDF 문서에서 문단별로 텍스트 추출 (표는 제외)
    """
    doc_list = []
    pdf_doc = fitz.open(pdf_path)  # PyMuPDF로 PDF 열기

    with pdfplumber.open(pdf_path) as pdf:
        for page_num in range(len(pdf_doc)):
            page = pdf_doc[page_num]

            # 텍스트 추출 (문단 단위로 구분)
            text = page.get_text("text")
            paragraphs = text.split("\n\n")  # 빈 줄로 문단 구분

            # 표 추출
            tables = pdf.pages[page_num].extract_tables()
            table_texts = []
            if tables:
                for table in tables:
                    df = pd.DataFrame(table[1:], columns=table[0])  # 첫 행을 컬럼으로 설정
                    table_texts.append(df.to_string())
            else:
                table_texts = []  # 표가 없는 경우 빈 리스트

            # 문서 객체 생성 (문단별로 추가)
            for para in paragraphs:
                doc = Document(
                    page_content=para + "\n\n" + "\n\n".join(table_texts),  # 문단 + 표 포함
                    metadata={"page": page_num + 1}  # 페이지 정보 포함
                )
                doc_list.append(doc)

    return doc_list

# 4. 텍스트 벡터화 (SentenceTransformer 사용)
def text_to_vector(text):
    """
    텍스트를 벡터로 변환 (SentenceTransformer 사용)
    """
    vector = embedding_model.encode(text)  # 바로 numpy 배열 반환
    return vector


# 5. FAISS DB에 새로운 문서 추가 (기존 인덱스 ID와 겹치지 않게 추가)
def add_pdf_to_faiss(pdf_path, faiss_index, docstore, index_to_docstore_id):
    """
    주어진 PDF에서 문단을 추출하고 FAISS 인덱스에 추가 (진행 상황 출력)
    """
    # 기존 FAISS 인덱스에 추가된 벡터 수 파악 (새로운 벡터 ID가 기존과 겹치지 않도록 함)
    existing_vector_count = faiss_index.ntotal
    print(f"기존 FAISS 인덱스의 벡터 수: {existing_vector_count}")

    # PDF에서 문단별로 텍스트 추출
    documents = extract_text_paragraphs(pdf_path)
    
    # 문서의 총 문단 수
    total_paragraphs = len(documents)
    
    # 문단 벡터화
    vectors = []
    for idx, doc in enumerate(documents):
        # 진행 상황 출력
        progress = (idx + 1) / total_paragraphs * 100
        print(f"진행 상황: {progress:.2f}% - {idx+1}/{total_paragraphs} 문단 처리 중...")
        
        # 문단 벡터화
        vector = text_to_vector(doc.page_content)
        vectors.append(vector)

        # 문서 ID와 문서 내용 저장 (docstore)
        doc_id = existing_vector_count + len(vectors) - 1  # docstore에 새 문서의 ID는 마지막 인덱스 + 1
        docstore[doc_id] = doc
        index_to_docstore_id[existing_vector_count + idx] = str(doc_id)  # 벡터 ID와 문서 ID 연결

    # 벡터를 FAISS DB에 추가
    vectors = np.array(vectors).astype('float32')
    
    # 기존 FAISS 인덱스에 벡터 추가
    faiss_index.add(vectors)

    # FAISS DB 저장
    faiss.write_index(faiss_index, "./faiss_artworks_0303/index.faiss")
    print(f"FAISS index updated and saved to './faiss_artworks_0303/index.faiss'.")

# 6. 전체 실행 흐름
if __name__ == "__main__":
    # 벡터 차원 크기 (SentenceTransformer 모델에 맞는 차원)
    embedding_dim = 768  # 예시: KURE-v1 모델은 768차원

    # 기존 FAISS DB 로드 (기존 인덱스가 없으면 새로 생성)
    faiss_index = load_existing_faiss_db("./faiss_artworks_0303/index.faiss", embedding_dim)

    # 문서 저장소 초기화 (문서 ID와 내용 매핑)
    docstore = {}
    index_to_docstore_id = {i: i for i in range(len(docstore))}

    # 새로운 PDF 파일 경로 지정
    pdf_path = './PDF/dataset/artworks_10.pdf'

    # 새 PDF 문서를 기존 FAISS DB에 추가
    add_pdf_to_faiss(pdf_path, faiss_index, docstore, index_to_docstore_id)

    # 새로운 문서가 docstore에 잘 추가되었는지 확인
    print(f"현재 문서 저장소에 저장된 문서 개수: {len(docstore)}")
    print(f"첫 번째 문서 ID: {list(docstore.keys())[0]}")
    print(f"첫 번째 문서 내용: {docstore[list(docstore.keys())[0]].page_content[:100]}...")  # 첫 100글자만 출력


In [ ]:
len(docstore)

In [ ]:
print(f"문서 ID: {list(docstore.keys())[10]}")
print(f"첫 번째 문서 내용: {docstore[list(docstore.keys())[10]].page_content[:100]}...")

In [ ]:
query = "부산 전차 구조안전진단 및 보존처리에서 차량 구조는 어떻게 되었는가?"

In [ ]:
response = chain.invoke({"question": query})
print(response)

In [ ]:
# 매핑된 문서 ID 확인
for vector_id in range(faiss_index.ntotal):
    if vector_id in index_to_docstore_id:
        doc_id = index_to_docstore_id[vector_id]
        print(f"벡터 ID: {vector_id}, 문서 ID: {doc_id}, 문서 내용: {docstore[doc_id].page_content[:100]}...")
    else:
        print(f"벡터 ID: {vector_id}에 대한 문서 ID가 매핑되지 않았습니다.")
